In [7]:
#integrated. step wise change. modfications based on each extremum points
# works better_Main

from pyfirmata import Arduino, SERVO
from time import sleep
import keyboard
import time
import numpy as np
import pandas as pd
import os
import cv2 as cv
from threading import Thread
import datetime

def servofunction():
    
    global deltamax, deltamin, MAX, MIN, step, calculations, calfreq, step, sleep_time, emerexit, freqcalculations, emerexit2, emerexit3, emerexit4
    
    #port = 'COM5'
    #pin = 9
    board = Arduino('COM5')

    #board.digital[pin].mode = SERVO

    servo = board.get_pin('d:9:s')

    end = False
    avefeq = True
    freqcalculations = False
    
    
    deltamax = 0
    deltamin = 0
    calculations = False
    #sleep_time = float(input("sleep time (0.078 for 1 Hz frequency):"))
    #MIN = int(input("what is the minimum angle:"))
    minang = MIN + 90
    minanglist = np.array(minang)
    #MAX = int(input("what is the maximum angle:"))
    maxang = MAX + 90
    maxanglist = np.array(maxang)
    
    CBLUE = '\033[34m'
    CGREEN = '\033[92m'
    CEND = '\033[0m'
    CRED = '\033[91m'
    
    t1totalcal = True
    t2totalcal = True
    
    step = 5
    #step = int(input("what is the step angle:"))
    
    sleep_time = 0.078
    #sleep_time = float(input("sleep time (0.078 for 1 Hz frequency):"))

    def rotateServo(angle):
        global sleep_time
        servo.write(angle)
        sleep(sleep_time)

    print("                 Servo ready to start")
    while True:
        if keyboard.is_pressed("r"):
            print("Servo Starts")
            for i in range(90, minang, -step):
                    rotateServo(i)  

            #t = np.array([time.time()])
            if calfreq == True:
                frequency = np.ones(1)

            loopN = 1

            while True:

                if emerexit == True:
                    rotateServo(90)
                    break
                
                if emerexit4 == True:
                    rotateServo(90)
                    break
                    
                if emerexit5 == True:
                    rotateServo(90)
                    break
                
                step = cv.getTrackbarPos("step", "HSV value for mask")
                if step == 0:
                    step = 1
                sleep_time = (cv.getTrackbarPos("sleep_time", "HSV value for mask"))/1000
                if sleep_time < 0.01:
                    sleep_time = 0.01


                if deltamax > 3.5:
                    maxang = maxang + 4
                elif deltamax > 2.5:
                    maxang = maxang + 3
                elif deltamax > 1.5:
                    maxang = maxang + 2
                elif deltamax > 0.5:
                    maxang = maxang + 1
                elif deltamax > -0.5:
                    pass
                elif deltamax > -1.5:
                    maxang = maxang - 1
                elif deltamax > -2.5:
                    maxang = maxang - 2
                elif deltamax > -3.5:
                    maxang = maxang - 3
                else:
                    maxang = maxang - 4
                if deltamin > 3.5:
                    minang = minang + 4
                elif deltamin > 2.5:
                    minang = minang + 3
                elif deltamin > 1.5:
                    minang = minang + 2
                elif deltamin > 0.5:
                    minang = minang + 1
                elif deltamin > -0.5:
                    pass
                elif deltamin > -1.5:
                    minang = minang - 1
                elif deltamin > -2.5:
                    minang = minang - 2
                elif deltamin > -3.5:
                    minang = minang - 3
                else:
                    minang = minang - 4

                #print("\n SERVO Loop #%d" % loopN,"   \n")
                if calculations == True:
                    print(CBLUE + "max ang =", maxang, "   min ang =", minang, "" + CEND)
                    maxanglist = np.append(maxanglist, maxang)
                    minanglist = np.append(minanglist, minang)

                if maxang > 140 or minang < 30:
                    emerexit3 = True
                    
                if emerexit3 == True:
                    rotateServo(90)
                    print("Servo touched the device")
                    break
                    
                if maxang == minang or maxang < minang:
                    print("ERROR")
                    break

                if calfreq == True:
                    t1=time.time()
                
                if t1totalcal == True:
                    t1total = time.time()
                    t1totalcal = False
                
                for i in range(minang, maxang, step):
                    rotateServo(i)

                    #print(count, "applied angels =", i, " actual angel =", servo.read(), "time =", t[count])

                    if keyboard.is_pressed("q"):
                        cp = servo.read()
                        if cp < 90:
                            #print(cp)
                            for j in range (cp, 91, step):
                                rotateServo(j)
                        elif cp > 90:
                            #print(cp)
                            for j in range (cp, 89, -step):
                                rotateServo(j)
                        else:
                            pass
                        #print("q is pressed 60 to 120")
                        end = True
                        break
                if end == True:
                        print("press ", CRED + "E" + CEND," to stop servo")
                        break
                for i in range(maxang, minang-1, -step):
                    rotateServo(i)
                    #print(count, "applied angels =", i, " actual angel =", servo.read(), "time =", t[count])

                    if keyboard.is_pressed("q"):
                        cp = servo.read()
                        if cp > 90:
                            #print(cp)
                            for j in range (cp, 89, -step):
                                rotateServo(j)
                        elif cp < 90:
                            #print(cp)
                            for j in range (cp, 91, step):
                                rotateServo(j)
                        else:
                            pass
                        #print("q is pressed 120 to 60")
                        end = True
                        break
                if calfreq == True:
                    t2=time.time()   
                if end == True:
                        print("press ", CRED + "E" + CEND," to stop servo")
                        break
                #a = int(((((maxang-minang)*2)/10)+1))
                if calfreq == True and freqcalculations == True:
                    fre = 1/(t2-t1)
                    frequency = np.append(frequency, fre)
                    #print("t loop begin=", t[count-13]," t loop end=", t[count-1])
                    print(CGREEN + "Frequency= ", frequency[-1], "" + CEND)
                    loopN = loopN + 1

        if calfreq == True and calculations == True and avefeq == True:
            dffrequency = pd.DataFrame(frequency)
            dffrequency.to_excel("frequency.xlsx", float_format = "%.3f", header = False)
            mF = (sum(frequency)-1)/(frequency.size-1)
            print("\n the average Frequency = ", mF)
            
        if calculations == True and avefeq == True:
            dfmaxanglist = pd.DataFrame(maxanglist)
            dfmaxanglist.to_excel("max angs on servo.xlsx", float_format = "%.3f", header = False)
            dfminanglist = pd.DataFrame(minanglist)
            dfminanglist.to_excel("min angs on servo.xlsx", float_format = "%.3f", header = False)
            avefeq = False
        
        if t2totalcal == True and t1totalcal == False:
            t2total = time.time()
            t2totalcal = False
            ttotal = t2total - t1total
            wholetime = str(datetime.timedelta(seconds=ttotal))
            print("The whole process time was ", wholetime, " (Hours:Minutes:Seconds)")
        
        if keyboard.is_pressed("e") or emerexit == True or emerexit2 == True or emerexit3 == True or emerexit4 == True or emerexit5 == True:
            #print("servo about to stop")
            board.exit()
            break
    print("servo stops")
    


                
                
                
    
def monitorfunction():
    
    global rect,startPoint,endPoint, assign, MAX, MIN, deltamax, deltamin, calculations, recvid, step, sleep_time, emerexit, freqcalculations, emerexit2, emerexit3, emerexit4, emerexit5
    
    #clear the folder for the frames:
    #dir = 'live_frames'
    #for f in os.listdir(dir):
     #   os.remove(os.path.join(dir, f))

    #live video:
    livevid = cv.VideoCapture(cv.CAP_DSHOW)
    
    livevid.set(cv.CAP_PROP_FRAME_WIDTH, 1920)
    livevid.set(cv.CAP_PROP_FRAME_HEIGHT, 1080)

    if recvid == True:
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        savef = cv.VideoWriter('source.avi', fourcc, 30, (640,  480))
        
    def empty(a):
        pass
    
    def sething(event,x,y,flags,params):
        global rect, hx, hy
        if event == cv.EVENT_LBUTTONDOWN:
                #rect = (x, y)
                hx = x
                hy = y
                #print("Hinge coordinates are (", hx, ", ", hy, ")")
                print("hinge point is selected: (", hx, ", ", hy, ")")
                print("Now press S to continue:")

    cv.namedWindow("HSV value for mask")
    cv.resizeWindow("HSV value for mask", 640,450)
    cv.createTrackbar("HUE Min", "HSV value for mask", 95, 179, empty)
    cv.createTrackbar("HUE Max", "HSV value for mask", 179, 179, empty)
    cv.createTrackbar("SAT Min", "HSV value for mask", 95, 255, empty)
    cv.createTrackbar("SAT Max", "HSV value for mask", 255, 255, empty)
    cv.createTrackbar("VALUE Min", "HSV value for mask", 95, 255, empty)
    cv.createTrackbar("VALUE Max", "HSV value for mask", 255, 255, empty)
    cv.createTrackbar("step", "HSV value for mask", 3, 20, empty)
    cv.createTrackbar("sleep_time", "HSV value for mask", 30, 100, empty)
    
    tracker = cv.TrackerCSRT_create()
    
    #timeout = time.time() + 5
    
    print("Adjust the filters and choose the hinge point on the original video:")
    while True:
        
        if keyboard.is_pressed("Esc"):
            emerexit5 = True
            
        if emerexit5 == True:
            break
        
        liveret, livef = livevid.read()
        livef = cv.medianBlur(livef,5)    
        livehsv = cv.cvtColor(livef, cv.COLOR_BGR2HSV)

        h_min = cv.getTrackbarPos("HUE Min", "HSV value for mask")
        h_max = cv.getTrackbarPos("HUE Max", "HSV value for mask")
        s_min = cv.getTrackbarPos("SAT Min", "HSV value for mask")
        s_max = cv.getTrackbarPos("SAT Max", "HSV value for mask")
        v_min = cv.getTrackbarPos("VALUE Min", "HSV value for mask")
        v_max = cv.getTrackbarPos("VALUE Max", "HSV value for mask")

        lower_range = np.array([h_min, s_min, v_min])
        upper_range = np.array([h_max, s_max, v_max])

        livemask = cv.inRange(livehsv, lower_range, upper_range)
        livemask = cv.medianBlur(livemask,5)
        cv.imshow('live mask:', livemask)
        cv.imshow("live video:", livef)
        
        #cv.setMouseCallback('live mask:', sething)
        cv.setMouseCallback("live video:", sething)
        
        if cv.waitKey(33) == ord('s'):
            break
        #if time.time() > timeout:
         #   break
    #print("choose ROI for hing point")
    #hbbox = cv.selectROI("live mask:", livemask, False)
    #hx = 231
    #hy = 10
    #print("Hinge coordinates are (325, 6)")
    if emerexit5 == False:
        print("choose object for tracking")
        cv.rectangle(livemask, (hx-2, hy-2), (hx+2, hy+2), (255,0,255), 3, 1)
        cv.line(livemask, (hx, hy), (hx, 480), (0,255,0), 3, 1)
        cv.line(livemask, (hx, hy), (hx, 480), (255,0,255), 1, 1)
        cv.line(livemask, (hx-28, hy+350), (hx+28, hy+350), (255,0,255), 2, 1)
        #cv.rectangle(livef, (323, 4), (327, 8), (255,0,255), 3, 1)
        bbox = cv.selectROI("live mask:", livemask, False)
        print("Coordinates for the center of the tracking object are (", bbox[0]+(bbox[2]/2),", ", bbox[1]+(bbox[3]/2),")")
    
    
        aa = hx-(bbox[0]+(bbox[2]/2))
        if aa<6.5 and aa>-6.5:
            print("The hinge and tracking center are aligned")
            pass
        else:
            print("The hinge and tracking center are NOT aligned")
            emerexit2 = True
        
    
        tracker.init(livemask, bbox) 

        def drawbox(livemask, bbox):
            global cx, cy
            x, y, w, h= int(bbox[0]), (bbox[1]), (bbox[2]), (bbox[3])
            cv.rectangle(livemask, (x, y), ((x+w), (y+h)), (255,0,255), 3, 1)
            cv.putText(livef, "Tracking", (75,75), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            cx=x+w/2
            cy=y+h/2

        #def findh(livemask, hbbox):
         #   global hx, hy
          #  u, v, n, m= int(hbbox[0]), (hbbox[1]), (hbbox[2]), (hbbox[3])
           # hx=u+n/2
            #hy=v+m/2

        #ROIs selection:
        #ROIs = False
        start = False


        startPoint = False
        #endPoint = False
        assign = True


        #saving frames and finding points and calculations::
        label1 = True
        label2 = True
        count = 0
        calculations = False
        angles = np.zeros(1)
        #hingepoint = False
        #checkpoint = True
        #Freqcal = True
        #freqframe = np.zeros(1)
        #frequency = np.ones(1)

        #detect local peaks:
        maxpeaks = np.array([0])
        minpeaks = np.array([0])
        findmax = False
        findmin = True
        avemax = np.array([30])
        removeavemax0 = True
        avemin = np.array([-30])
        removeavemin0 = True
        meanmax = False
        meanmin = False
        loopnum = 1
        loopmax = False
        loopmin = False


        CYELLOW  = '\033[33m'
        CRED = '\033[91m'
        CEND = '\033[0m'

        if emerexit2 == False:
            print("Press r to initiate the servo:")
            
        size = (int(livevid.get(cv.CAP_PROP_FRAME_WIDTH)), int(livevid.get(cv.CAP_PROP_FRAME_HEIGHT)))
        
    while True:
        if emerexit == True:
            break
        if emerexit2 == True:
            break
        if emerexit3 == True:
            break
        if emerexit4 == True:
            break
            
        if keyboard.is_pressed("Esc"):
            emerexit5 = True
            
        if emerexit5 == True:
            break
        
        liveret, livef = livevid.read()
        if liveret == True:

            livef = cv.medianBlur(livef,5)    
            livehsv = cv.cvtColor(livef, cv.COLOR_BGR2HSV)

            h_min = cv.getTrackbarPos("HUE Min", "HSV value for mask")
            h_max = cv.getTrackbarPos("HUE Max", "HSV value for mask")
            s_min = cv.getTrackbarPos("SAT Min", "HSV value for mask")
            s_max = cv.getTrackbarPos("SAT Max", "HSV value for mask")
            v_min = cv.getTrackbarPos("VALUE Min", "HSV value for mask")
            v_max = cv.getTrackbarPos("VALUE Max", "HSV value for mask")

            lower_range = np.array([h_min, s_min, v_min])
            upper_range = np.array([h_max, s_max, v_max])

            livemask = cv.inRange(livehsv, lower_range, upper_range)
            livemask = cv.medianBlur(livemask,5)
            
            success, bbox = tracker.update(livemask)
            
            #TRACKING:
            
            if success:
                drawbox(livemask, bbox)
                #print(cx, ", ", cy)
            else:
                cv.putText(livef, "lost", (75,75), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                print(CRED + "Tracking object is Lost; EMERGNCY EXIT ACTIVATED.", "" + CEND)
                emerexit = True
           
            cv.imshow("live video:", livef)
            cv.imshow('live mask:', livemask)
            

                #Select ROIS:
            if start == False:
                if cv.waitKey(33) == ord('r'):
                    freqcalculations = True
                    start = True
                    

            #if ROIs == True and ROIex == False:

#                findh(livemask, hbbox)
 #               print("hing coordinates are:", hx, " ", hy)
  #              ROIex = True

#                if cv.waitKey(33) == ord('q'):
 #                   #print('im here1')
  #                  break        
                #cv.imshow('live mask:',livemask)       


            if start == True and calculations == False:
                if label1 == True:
                    print("Now press c to do calculations.")
                    label1 = False
                #cv.rectangle(livef, (psp[0], psp[1]), (pep[0], pep[1]), (0, 0, 255), 2)
                if cv.waitKey(33) == ord('c'):
                    print("Calculations are being done.")
                    calculations = True
                    #print('im here1')
            if calculations == True:
                
                #save video:
                if recvid == True:
                    if label2 == True:
                        print("Video is being recorded")
                        label2 = False
                    savef.write(livef)

                ang = np.rad2deg(np.arctan((hx-cx)/(cy-hy)))

                angles = np.append(angles, ang)

                
                if ang > 40 or ang < -40:
                    print("extreme angle is applied on the animal")
                    emerexit4 = True
                    
            
                #to find maximum angles:
                if findmax == True:
                    if angles[count] < angles[count-1]:
                        maxpeaks = np.append(maxpeaks, angles[count-1])
                        #print("\n relative max angle = ", maxpeaks[-1],"\n")
                        findmax =False
                        findmin = True
                        meanmax = True
                        
                        maxerror = True


                #to find minimum angles:
                if findmin == True:
                    if angles[count] > angles[count-1]:
                        minpeaks = np.append(minpeaks, angles[count-1])
                        #print("\n relative min angle = ", minpeaks[-1],"\n")
                        findmin = False
                        findmax = True
                        meanmin = True
                        
                        minerror = True

                #to report the mean max points so far:
                if meanmax == True:
                    if maxpeaks[-1] > 5:
                        avemax = np.append(avemax, maxpeaks[-1])
                        if removeavemax0 == True:
                            avemax = np.delete(avemax, 0)
                            removeavemax0 = False
                        ave_max_ang = np.average(avemax)
                        print("average max:", ave_max_ang)
                        #print("Maximum of this loop:", maxpeaks[-1])
                        deltamax = MAX - maxpeaks[-1]
                        #print("delta max =", deltamax)
                        meanmax = False
                        loopmax = True
                    elif maxerror == True:
                        print(CRED + "too low max angle is detected", maxpeaks[-1],"" + CEND)
                        maxerror = False
                        deltamax = 0
                #to report the mean min points so far:
                if meanmin == True:
                    if minpeaks[-1] < -5:
                        avemin = np.append(avemin, minpeaks[-1])
                        if removeavemin0 == True:
                            avemin = np.delete(avemin, 0)
                            removeavemin0 = False
                        ave_min_ang = np.average(avemin)
                        print("average min:", ave_min_ang)
                        #print("MINIMUM of this loop:", minpeaks[-1])
                        deltamin = MIN - minpeaks[-1]
                        #print("delta min =", deltamin)
                        meanmin = False
                        loopmin = True
                    elif minerror == True:
                        print(CRED + "too low min angle is detected:", minpeaks[-1],"" + CEND)
                        minerror = False
                        deltamax = 0
                if loopmax == True and loopmin == True:
                    print(CYELLOW + "monitored Loop# %d:" % loopnum, "" + CEND)
                    loopnum = loopnum + 1
                    loopmax = False
                    loopmin = False
                count = count + 1
                
                    
                if cv.waitKey(33) == ord('q'):
                    #print('im here2')
                    break 

            pass

        else:
            print("Video is not availble anymore")
            break        


    livevid.release()
    print("Monitoring is stopped")
    #print("count=", count)
    if recvid == True:
        savef.release()
        #savem.release()
    cv.destroyAllWindows()
    
    print('size:', size)

    dfangles = pd.DataFrame(angles)
    dfangles.to_excel("angles.xlsx", float_format = "%.3f", header = False)
    dfangles = pd.DataFrame(avemin)
    dfangles.to_excel("min points.xlsx", float_format = "%.3f", header = False)
    dfangles = pd.DataFrame(avemax)
    dfangles.to_excel("max points.xlsx", float_format = "%.3f", header = False)

In [8]:
#MIN = int(input("what is the minimum angle:"))
MIN = -30
#MAX = int(input("what is the maximum angle:"))
MAX = 30

emerexit = False
emerexit2 = False
emerexit3 = False
emerexit4 = False
emerexit5 = False
#emerexit: if tracker is lost during the process
#emerexit2: if hinged point and tracker are not aligned at the begining
#emerexit3: if servo touched the device during the process
#emerexit4: if too large angle is applied on the animal
#emerexit5: user controlled emerexit

calfreqyn = input("Do you want to calculate the frequency? (yes/no):")

if calfreqyn == "yes":
    calfreq = True
else:
    calfreq = False
    
recvidyn = input("Do you want to record the video? (yes/no):")

if recvidyn == "yes":
    recvid = True
else:
    recvid = False

Thread(target=monitorfunction, args=()).setDaemon

Thread(target=monitorfunction, args=()).start() 

time.sleep(5)
    
Thread(target=servofunction, args=()).start()    
    


#print("Threads are started")

Do you want to calculate the frequency? (yes/no):no
Do you want to record the video? (yes/no):no
Adjust the filters and choose the hinge point on the original video:
                 Servo ready to start
hinge point is selected: ( 1118 ,  164 )
Now press S to continue:
choose object for tracking
Coordinates for the center of the tracking object are ( 1121.0 ,  468.0 )
The hinge and tracking center are aligned
Press r to initiate the servo:
Servo Starts
Now press c to do calculations.
Calculations are being done.
too low min angle is detected: -3.366460663429801 
too low max angle is detected 0.0 
average min: -19.33480853782641
average max: 17.81219313571569
monitored Loop# 1: 
max ang = 124    min ang = 56 
average min: -19.878570781274924
average max: 19.32443425032961
monitored Loop# 2: 
max ang = 128    min ang = 52 
average min: -20.08204063760802
average max: 20.412556803510807
monitored Loop# 3: 
max ang = 132    min ang = 48 
average min: -21.455296693957237
average max: 21.458